In [17]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from ras_method import ras_method
import subprocess
data_path = os.path.join('..','Data')

import warnings
warnings.filterwarnings('ignore')

In [18]:
ind_mapper = pd.read_excel(os.path.join(data_path,'database_tables','input','sh_cou_06_16.xls'),
                          sheet_name='ind_mapper',header=None)
ind_mapper = dict(zip(ind_mapper[0],ind_mapper[1]))

In [19]:
com_mapper = pd.read_excel(os.path.join(data_path,'database_tables','input','sh_cou_06_16.xls'),
                          sheet_name='com_mapper',header=None)
com_mapper = dict(zip(com_mapper[0],['P_'+x for x in com_mapper[1]]))

In [20]:
reg_mapper = pd.read_excel(os.path.join(data_path,'database_tables','input','sh_cou_06_16.xls'),
                          sheet_name='reg_mapper',header=None)
reg_mapper = dict(zip(reg_mapper[0], reg_mapper[1]))

In [21]:
sectors = [chr(i) for i in range(ord('A'),ord('P')+1)]

### Load supply table and aggregate 

In [22]:
sup_table_in = pd.read_excel(os.path.join(data_path,'economic_IO_tables','input','sh_cou_06_16.xls'),
                          sheet_name='Mat Oferta pb',skiprows=2,header=[0,1],index_col=[0,1],nrows=271)
sup_table_in = sup_table_in.drop('Total',level=0,axis=1)
sup_table = sup_table_in.copy()

In [23]:
sup_table.columns = sup_table.columns.get_level_values(0)
sup_table.columns = sup_table.columns.map(ind_mapper)
sup_table = sup_table.T.groupby(level=0,axis=0).sum()
sup_table.columns = sup_table.columns.get_level_values(0)
sup_table.columns = sup_table.columns.map(com_mapper)
sup_table = sup_table.T.groupby(level=0,axis=0).sum()
#sup_table = sup_table.astype(int)

In [24]:
sup_table

,"A - AGRICULTURA ,GANADERIA,CAZA Y SILVICULTURA.",B - PESCA,C - EXPLOTACIÓN DE MINAS Y CANTERAS,D - INDUSTRIA MANUFACTURERA,"E - ELECTRICIDAD, GAS Y AGUA",F - CONSTRUCCIÓN,"G - COMERCIO MAYORISTA, MINORISTA Y REPARACIONES",H - HOTELES Y RESTAURANTES,I - TRANSPORTE Y COMUNICACIONES,J - INTERMEDIACIÓN FINANCIERA,"K - ACTIVIDADES EINMOBILIARIAS, EMPRESARIALES Y DE ALQUILER",L - ADMINISTRACIÓN PÚBLICA Y DEFENSA; PLANES DE SEGURIDAD SOCIAL DE AFILIACIÓN OBLIGATORIA,M - ENSEÑANZA,N - SERVICIOS SOCIALES Y DE SALUD,"O - OTRAS ACTIVIADADES DE SERVICIOS COMUNITARIAS, SOCIALES Y PERSONALES",P - HOGARES PRIVADOS CON SERVICIO DOMÉSTICO
"P_ACTIVIDADES INMOBILIARIAS, EMPRESARIALES Y DE ALQUILER",0.000000e+00,0.000000e+00,4.682709e+04,5.496790e+05,0.0,0.0,9.366056e+05,8.117000e+04,0.000000e+00,0.000000e+00,4.265877e+07,0.000000e+00,0.000000e+00,0.000000e+00,1.778169e+05,0.000000e+00
P_ADMINISTRACIÓN PÚBLICA Y DEFENSA,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.634524e+07,0.000000e+00,3.263913e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
"P_AGRICULTURA, GANADERÍA, CAZA Y SILVICULTURA",6.972756e+07,2.110688e+04,0.000000e+00,2.844877e+07,0.0,0.0,8.401602e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
P_COMERCIO,0.000000e+00,0.000000e+00,7.480000e+04,1.422609e+07,0.0,0.0,8.332742e+07,2.571310e+05,1.515429e+07,4.321686e+06,2.602928e+07,0.000000e+00,0.000000e+00,0.000000e+00,1.453163e+07,3.042849e+06
P_CONSTRUCCIÓN,2.121366e+06,0.000000e+00,4.378649e+06,5.548503e+05,0.0,42662909.0,5.987146e+04,0.000000e+00,0.000000e+00,0.000000e+00,2.440728e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
P_EDUCACIÓN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.052639e+07,0.000000e+00,0.000000e+00,0.000000e+00
"P_ELECTRICIDAD, GAS Y AGUA",0.000000e+00,0.000000e+00,1.686294e+04,1.499814e+06,20890231.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
P_EXPLOTACIÓN DE MINAS Y CANTERAS,0.000000e+00,0.000000e+00,3.463957e+07,1.016211e+07,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
P_HOTELES Y RESTAURANTES,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,1.014779e+05,1.756677e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
P_INDUSTRIA MANUFACTURERA,0.000000e+00,1.477492e+06,7.104148e+03,2.328190e+08,0.0,0.0,8.048386e+04,1.539900e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


### Load use table

In [25]:
use_table = pd.read_excel(os.path.join(data_path,'economic_IO_tables','input','sh_cou_06_16.xls'),
                          sheet_name='Mat Utilizacion pc',skiprows=2,header=[0,1],index_col=[0,1],nrows=271)

basic_prod_prices = use_table[[#'PRODUCCION NACIONAL A PRECIOS BASICOS',
                               'IMPORTACIONES  (CIF a nivel de producto y FOB a nivel total)',
                               'AJUSTE CIF/FOB DE LAS IMPORTACIONES','DERECHOS DE IMPORTACION',
                               'IMPUESTOS A LOS PRODUCTOS NETOS DE SUBSIDIOS','MARGENES DE COMERCIO',
                               'MARGENES DE TRANSPORTE','IMPUESTO AL VALOR AGREGADO NO DEDUCIBLE',
                               #'OFERTA TOTAL A PRECIOS DE  COMPRADOR'
                                ]]*-1

use_table = use_table.drop(['PRODUCCION NACIONAL A PRECIOS BASICOS',
                            'IMPORTACIONES  (CIF a nivel de producto y FOB a nivel total)',
                            'AJUSTE CIF/FOB DE LAS IMPORTACIONES','DERECHOS DE IMPORTACION',
                            'IMPUESTOS A LOS PRODUCTOS NETOS DE SUBSIDIOS','MARGENES DE COMERCIO',
                            'MARGENES DE TRANSPORTE','IMPUESTO AL VALOR AGREGADO NO DEDUCIBLE',
                            'OFERTA TOTAL A PRECIOS DE  COMPRADOR','UTILIZACION INTERMEDIA',
                            'UTILIZACION FINAL','DEMANDA TOTAL'],level=0,axis=1)

In [26]:
basic_prod_prices.columns = basic_prod_prices.columns.get_level_values(0)
basic_prod_prices = basic_prod_prices.T.groupby(level=0,axis=0).sum()
basic_prod_prices.columns = basic_prod_prices.columns.get_level_values(0)
basic_prod_prices.columns = basic_prod_prices.columns.map(com_mapper)
basic_prod_prices = basic_prod_prices.T.groupby(level=0,axis=0).sum()
basic_prod_prices = basic_prod_prices.astype(int)

use_table.columns = use_table.columns.get_level_values(0)
use_table.columns = use_table.columns.map(ind_mapper)
use_table = use_table.T.groupby(level=0,axis=0).sum()
use_table.columns = use_table.columns.get_level_values(0)
use_table.columns = use_table.columns.map(com_mapper)
use_table = use_table.T.groupby(level=0,axis=0).sum()
#use_table = use_table.astype(int)

use_table= pd.concat([use_table,basic_prod_prices],axis=1)

### Create Industry-Industry IO table

In [27]:
# GET VARIABLES
x = np.array(sup_table.sum(axis=0)) # total production on industry level
g = np.array(sup_table.sum(axis=1)) # total production on product level
F = use_table.iloc[:16,16:].sum(axis=1)

#Numpify
Sup_array = np.asarray(sup_table.iloc[:16,:16]) # numpy array of supply matrix
Use_array = np.asarray(use_table.iloc[:16,:16]) # numpy array of use matrix

g_diag_inv = np.linalg.inv(np.diag(g)) # inverse of g (and diagolinized)
x_diag_inv = np.linalg.inv(np.diag(x)) # inverse of x (and diagolinized)

# Calculate the matrices
B = np.dot(Use_array,x_diag_inv) # B matrix (U*x^-1)
D = np.dot(Sup_array.T,g_diag_inv) # D matrix (V*g^-1)
I_i = np.identity((len(x))) # Identity matrix for industry-to-industry

In [28]:
# Inverse for industry-to-industry
A_ii = np.dot(D,B)
F_ii = np.dot(D,F)/1e6
IDB_inv = np.linalg.inv((I_i-np.dot(D,B))) # (I-DB)^-1 

# And canclulate sum of industries
ind = np.dot(IDB_inv,np.dot(D,F)/1e6) # (I-DB)^-1 * DF

In [29]:
# split FD in local, import and export
LFD = np.dot(D,use_table.iloc[:16,[16,18,19,21,22,23,24]].sum(axis=1) )/1e6
Exp = np.dot(D,use_table.iloc[:16,17])/1e6
Imp = np.dot(D,use_table.iloc[:16,20])/1e6

In [30]:
IO_ARG = pd.concat([pd.DataFrame(np.dot(A_ii,np.diag(ind))),pd.DataFrame(LFD),pd.DataFrame(Exp)],axis=1)

In [14]:
IO_ARG = pd.concat([pd.DataFrame(np.dot(A_ii,np.diag(ind))),pd.DataFrame(LFD),pd.DataFrame(Exp)],axis=1)
IO_ARG.columns = list(use_table.columns[:18])
IO_ARG.index = list(use_table.columns[:16])
VA = np.array(list(ind)+[0,0])-np.array(IO_ARG.sum(axis=0))
IMP = np.array(list(Imp*-1)+[0,0])
VA[-2:] = 0
IO_ARG.loc['ValueA'] = VA
IO_ARG.loc['Imports'] = IMP
IO_ARG[IO_ARG < 1e-5] = 0

In [32]:
list(use_table.columns[:18])

['A -  AGRICULTURA ,GANADERIA,CAZA Y SILVICULTURA. ',
 'B - PESCA',
 'C - EXPLOTACIÓN DE MINAS Y CANTERAS',
 'D - INDUSTRIA MANUFACTURERA',
 'E - ELECTRICIDAD, GAS Y AGUA',
 'F - CONSTRUCCIÓN',
 'G - COMERCIO MAYORISTA, MINORISTA Y REPARACIONES',
 'H - HOTELES Y RESTAURANTES',
 'I - TRANSPORTE Y COMUNICACIONES',
 'J - INTERMEDIACIÓN FINANCIERA',
 'K - ACTIVIDADES EINMOBILIARIAS, EMPRESARIALES Y DE ALQUILER',
 'L - ADMINISTRACIÓN PÚBLICA Y DEFENSA; PLANES DE SEGURIDAD SOCIAL DE AFILIACIÓN OBLIGATORIA',
 'M - ENSEÑANZA',
 'N - SERVICIOS SOCIALES Y DE SALUD',
 'O - OTRAS ACTIVIADADES DE SERVICIOS COMUNITARIAS, SOCIALES Y PERSONALES',
 'P - HOGARES PRIVADOS CON SERVICIO DOMÉSTICO',
 'UTILIZACION FINAL',
 'U_EXPORTACIONES']

In [15]:
IO_ARG.sum().sum()

1413.200809596147

## First iteration, no trade to determine total regional input and output

### Load provincial data

In [14]:
prov_data = pd.read_excel(os.path.join(data_path,'economic_IO_tables','input','PIB_provincial_06_17.xls'),sheet_name='VBP',
                         skiprows=3,index_col=[0],header=[0],nrows=71)
prov_data = prov_data.loc[[x.isupper() for x in prov_data.index],:]
prov_data.columns = [x.replace(' ','_') for x in ['Ciudad de Buenos Aires', 'Buenos Aires', 'Catamarca', 'Cordoba',
       'Corrientes', 'Chaco', 'Chubut', 'Entre Rios', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquen', 'Rio Negro',
       'Salta', 'San Juan', 'San Luis', 'Santa Cruz', 'Santa Fe',
       'Santiago del Estero', 'Tucuman', 'Tierra del Fuego',
       'No distribuido', 'Total']]
region_names = list(prov_data.columns)[:-2]

In [15]:
prov_data.index = sectors+['TOTAL']

In [16]:
prov_data = prov_data.replace(0, 1)

### Create proxy data for first iteration

In [40]:
# proxy level 2
proxy_reg_arg = pd.DataFrame(prov_data.iloc[-1,:24]/prov_data.iloc[-1,:24].sum()).reset_index()
proxy_reg_arg['year'] = 2016
proxy_reg_arg = proxy_reg_arg[['year','index','TOTAL']]
proxy_reg_arg.columns = ['year','id','gdp']
proxy_reg_arg.to_csv(os.path.join(data_path,'mrio_analysis','proxy_reg_arg.csv'),index=False)

In [41]:
# proxy level 4
for iter_,sector in enumerate(sectors+['other1','other2']):
    if (sector is not 'other1') & (sector is not 'other2'):
        proxy_sector = pd.DataFrame(prov_data.iloc[iter_,:24]/prov_data.iloc[iter_,:24].sum()).reset_index()
        proxy_sector['year'] = 2016
        proxy_sector['sector'] = 'sec{}'.format(sector)
        proxy_sector = proxy_sector[['year','sector','index',sector]]
        proxy_sector.columns = ['year','sector','region','gdp']
        proxy_sector.to_csv(os.path.join(data_path,'mrio_analysis','proxy_sec{}.csv'.format(sector)),index=False)
    else:
        proxy_sector = pd.DataFrame(prov_data.iloc[-1,:24]/prov_data.iloc[-1,:24].sum()).reset_index()
        proxy_sector['year'] = 2016
        proxy_sector['sector'] = sector+'1'
        proxy_sector = proxy_sector[['year','sector','index','TOTAL']]
        proxy_sector.columns = ['year','sector','region','gdp']
        proxy_sector.to_csv(os.path.join(data_path,'mrio_analysis','proxy_{}.csv'.format(sector)),index=False)

In [42]:
def change_name(x):
    if x in sectors:
        return 'sec'+x
    elif x == 'other1':
        return 'other11'
    else:
        return 'other21'

In [43]:
# proxy level 18
mi_index = pd.MultiIndex.from_product([sectors+['other1','other2'], region_names, sectors+['other1','other2'], region_names],
                                     names=['sec1', 'reg1','sec2','reg2'])
for iter_,sector in enumerate(sectors+['other1','other2']):
    if (sector is not 'other1') & (sector is not 'other2'):
        proxy_trade = pd.DataFrame(columns=['year','gdp'],index= mi_index).reset_index()
        proxy_trade['year'] = 2016
        proxy_trade['gdp'] = 0
        proxy_trade = proxy_trade.query("reg1 != reg2")
        proxy_trade = proxy_trade.loc[proxy_trade.sec1 == sector]
        proxy_trade['sec1'] = proxy_trade.sec1.apply(change_name)
        proxy_trade['sec2'] = proxy_trade.sec2.apply(change_name)
        proxy_trade = proxy_trade[['year','sec1','reg1','sec2','reg2','gdp']]
        proxy_trade.columns = ['year','sector','region','sector','region','gdp']
        proxy_trade.to_csv(os.path.join(data_path,'mrio_analysis','proxy_trade_sec{}.csv'.format(sector)),index=False)    
    else:
        proxy_trade = pd.DataFrame(columns=['year','gdp'],index= mi_index).reset_index()
        proxy_trade['year'] = 2016
        proxy_trade['gdp'] = 0
        proxy_trade = proxy_trade.query("reg1 != reg2")    
        proxy_trade = proxy_trade.loc[proxy_trade.sec1 == sector]
        proxy_trade['sec1'] = proxy_trade.sec1.apply(change_name)
        proxy_trade['sec2'] = proxy_trade.sec2.apply(change_name)       
        proxy_trade = proxy_trade[['year','sec1','reg1','sec2','reg2','gdp']]
        proxy_trade.columns = ['year','sector','region','sector','region','gdp']
        proxy_trade.to_csv(os.path.join(data_path,'mrio_analysis','proxy_trade_{}.csv'.format(sector)),index=False)

### Create first version of MRIO for Argentina

In [21]:
p = subprocess.Popen(['D:\Dropbox\OIA\Argentina\Data\mrio_analysis\mrio_disaggregate', 'settings_notrade.yml'],
                     cwd=os.path.join(data_path, 'mrio_analysis'))
p.wait()

0

In [22]:
region_names_list = [item for sublist in [[x]*(len(sectors)+2) for x in region_names]
                     for item in sublist]

In [23]:
rows = ([x for x in sectors+['VA','IMP']])*len(region_names)
cols = ([x for x in sectors+['FD','EXP']])*len(region_names)

In [186]:
index_mi = pd.MultiIndex.from_arrays([region_names_list, rows], names=('region', 'row'))
column_mi = pd.MultiIndex.from_arrays([region_names_list, cols], names=('region', 'col'))

MRIO = pd.read_csv(os.path.join(data_path,'mrio_analysis','output1.csv'),header=None,index_col=None)
MRIO.index = index_mi
MRIO.columns = column_mi

# create predefined index and col, which is easier to read
sector_only = [x for x in sectors]*len(region_names)
col_only = ['FD']*len(region_names)

region_col = [item for sublist in [[x]*len(sectors) for x in region_names] for item in sublist] + \
    [item for sublist in [[x]*1 for x in region_names] for item in sublist]

column_mi_reorder = pd.MultiIndex.from_arrays(
    [region_col, sector_only+col_only], names=('region', 'col'))

# sum va and imports
valueA = MRIO.xs('VA', level=1, axis=0).sum(axis=0)
valueA.drop('FD', level=1,axis=0,inplace=True)
valueA.drop('EXP', level=1,axis=0,inplace=True)
imports = MRIO.xs('IMP', level=1, axis=0).sum(axis=0)
imports.drop('FD', level=1,axis=0,inplace=True)
imports.drop('EXP', level=1,axis=0,inplace=True)
FinalD = MRIO.xs('FD', level=1, axis=1).sum(axis=1)
FinalD.drop('VA', level=1,axis=0,inplace=True)
FinalD.drop('IMP', level=1,axis=0,inplace=True)
Export = MRIO.xs('EXP', level=1, axis=1).sum(axis=1)
Export.drop('VA', level=1,axis=0,inplace=True)
Export.drop('IMP', level=1,axis=0,inplace=True)

output_new = MRIO.copy()

#MRIO.drop('FD', level=1,axis=1,inplace=True)
#MRIO.drop('EXP', level=1,axis=1,inplace=True)
#MRIO.drop('VA', level=1,axis=0,inplace=True)
#MRIO.drop('IMP', level=1,axis=0,inplace=True)

#output_new = pd.concat([MRIO,pd.DataFrame(FinalD,columns=[('ALL','FD')]),
#                        pd.DataFrame(Export,columns=[('ALL','EXP')])],axis=1)
#output_new = pd.concat([MRIO,pd.concat([pd.DataFrame(FinalD,columns=[('ALL','VA')]),
#                           pd.DataFrame(Export,columns=[('ALL','IMP')])],axis=1).T])


In [187]:
%%time
# convert to numpy matrix
X0 = MRIO.as_matrix()

# get sum of rows and columns
u = X0.sum(axis=1)
v = X0.sum(axis=0)

# and only keep T
v[:(len(u)-2)] = u[:-2]

# apply RAS method to rebalance the table
X1 = ras_method(X0, u, v, eps=1e-5,print_out=True)

0.5615995126531264
0.2409640267986286
0.14912878169535948
0.09117068295972808
0.05493917457227582
0.03304577936160391
0.01993426597448411
0.012081355261227555
0.00736011652925761
0.0045067533227654
0.002772578546328175
0.0017128668170276473
0.00106205977516316
0.0006712595227358342
0.0004614611409690106
0.00031631042127777675
0.0002163238665575662
0.0001476801122790583
0.00010067737021435264
6.855875485145546e-05
4.6646174667763773e-05
3.17154120805796e-05
2.1552030942562084e-05
1.4639246148195895e-05
Wall time: 340 ms


In [188]:
output_new = pd.DataFrame(X1)
output_new.index = index_mi
output_new.columns = column_mi

In [189]:
output_new.sum(axis=0).loc[:,'EXP'].sum()

986.0804561163997

## Second iteration, including trade

### Load od matrix

In [190]:
od_matrix_total = pd.DataFrame(pd.read_excel(os.path.join(data_path,'OD_data','province_ods.xlsx'),
                          sheet_name='total',index_col=[0,1],usecols =[0,1,2,3,4,5,6,7])).unstack(1).fillna(0)
od_matrix_total.columns.set_levels(['A','G','C','D','B','I'],level=0,inplace=True)
od_matrix_total.index = od_matrix_total.index.map(reg_mapper)
od_matrix_total = od_matrix_total.stack(0)
od_matrix_total.columns = od_matrix_total.columns.map(reg_mapper)
od_matrix_total = od_matrix_total.swaplevel(i=-2, j=-1, axis=0)
od_matrix_total = od_matrix_total.loc[:, od_matrix_total.columns.notnull()]

In [191]:
def est_trade_value(x,output_new,sector):
    if (sector is not 'other1') & (sector is not 'other2'):
        sec_output = output_new.sum(axis=1).loc[output_new.sum(axis=1).index.get_level_values(1) == sector].reset_index()
    else:
        sec_output = output_new.sum(axis=1).loc[output_new.sum(axis=1).index.get_level_values(1) == 'IMP'].reset_index()
    x['gdp'] = x.gdp*min(sec_output.loc[sec_output.region==x.reg1].values[0][2],sec_output.loc[sec_output.region==x.reg2].values[0][2])
#    x['gdp'] = x.gdp*(sec_output.loc[sec_output.region==x.reg1].values[0][2])
    return x

### Create proxy data for second iteration

In [192]:
# proxy level 14 
mi_index = pd.MultiIndex.from_product([sectors+['other1','other2'], region_names, region_names],
                                     names=['sec1', 'reg1','reg2'])

for iter_,sector in enumerate(tqdm(sectors+['other1','other2'])):
#    if sector in ['L','M','N','O','P']:
#        proxy_trade = pd.DataFrame(columns=['year','gdp'],index= mi_index).reset_index()
#        proxy_trade['year'] = 2016
#        proxy_trade['gdp'] = 0
#        proxy_trade = proxy_trade.query("reg1 != reg2")
#        proxy_trade = proxy_trade.loc[proxy_trade.sec1 == sector]
#        proxy_trade['sec1'] = ['sec'+x if x is not 'other' else 'other1' for x in proxy_trade.sec1 ]
#        proxy_trade = proxy_trade[['year','sec1','reg1','reg2','gdp']]
#        proxy_trade.columns = ['year','sector','region','region','gdp']
#        proxy_trade.to_csv(os.path.join(data_path,'mrio_analysis','proxy_trade_sec{}.csv'.format(sector)),index=False)
    if sector in ['A','G','C','D','B','I']:
        proxy_trade = (od_matrix_total.sum(level=1).divide(od_matrix_total.sum(level=1).sum(axis=1),axis='rows')).stack(0).reset_index()
        proxy_trade.columns = ['reg1','reg2','gdp']
        proxy_trade['year'] = 2016
        proxy_trade = proxy_trade.apply(lambda x: est_trade_value(x,output_new,sector),axis=1)
        proxy_trade['sec1'] = 'sec{}'.format(sector)
        proxy_trade = proxy_trade[['year','sec1','reg1','reg2','gdp']]
        proxy_trade.columns = ['year','sector','region','region','gdp']
        proxy_trade.to_csv(os.path.join(data_path,'mrio_analysis','proxy_trade14_sec{}.csv'.format(sector)),index=False)
    elif (sector is not 'other1') &  (sector is not 'other2') & (sector not in ['A','G','C','D','B','I']): # &  (sector not in ['L','M','N','O','P']):
        proxy_trade = (od_matrix_total.sum(level=1).divide(od_matrix_total.sum(level=1).sum(axis=1),axis='rows')).stack(0).reset_index()
        #proxy_trade[0].loc[(proxy_trade.origin_province == proxy_trade.destination_province)] = 0.9
        #proxy_trade[0].loc[~(proxy_trade.origin_province == proxy_trade.destination_province)] = 0.1
        proxy_trade.columns = ['reg1','reg2','gdp']
        proxy_trade['year'] = 2016
        proxy_trade = proxy_trade.apply(lambda x: est_trade_value(x,output_new,sector),axis=1)
        proxy_trade['sec1'] = 'sec{}'.format(sector)
        proxy_trade = proxy_trade[['year','sec1','reg1','reg2','gdp']]
        proxy_trade.columns = ['year','sector','region','region','gdp']
        proxy_trade.to_csv(os.path.join(data_path,'mrio_analysis','proxy_trade14_sec{}.csv'.format(sector)),index=False)

    else:
        proxy_trade = (od_matrix_total.sum(level=1).divide(od_matrix_total.sum(level=1).sum(axis=1),axis='rows')).stack(0).reset_index()
        proxy_trade.columns = ['reg1','reg2','gdp']
        proxy_trade['year'] = 2016
        proxy_trade = proxy_trade.apply(lambda x: est_trade_value(x,output_new,sector),axis=1)
        proxy_trade['sec1'] = sector+'1'
        proxy_trade = proxy_trade[['year','sec1','reg1','reg2','gdp']]
        proxy_trade.columns = ['year','sector','region','region','gdp']
        proxy_trade.to_csv(os.path.join(data_path,'mrio_analysis','proxy_trade14_{}.csv'.format(sector)),index=False)       

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [01:14<00:00,  3.93s/it]


In [193]:
# proxy level 18
mi_index = pd.MultiIndex.from_product([sectors+['other1','other2'], region_names, sectors+['other1','other2'], region_names],
                                     names=['sec1', 'reg1','sec2','reg2'])
for iter_,sector in enumerate(tqdm(sectors+['other1','other2'])):
    if (sector is not 'other1') & (sector is not 'other2'):
        proxy_trade = pd.DataFrame(columns=['year','gdp'],index= mi_index).reset_index()
        proxy_trade['year'] = 2016
        proxy_trade['gdp'] = 0
        proxy_trade = proxy_trade.query("reg1 != reg2")
        proxy_trade = proxy_trade.loc[proxy_trade.sec1 == sector]
        proxy_trade = proxy_trade.loc[proxy_trade.sec2.isin(['L','M','N','O','P'])]
        proxy_trade['sec1'] = proxy_trade.sec1.apply(change_name)
        proxy_trade['sec2'] = proxy_trade.sec2.apply(change_name) 
        
        proxy_trade = proxy_trade.query("reg1 == reg2")    

        proxy_trade = proxy_trade[['year','sec1','reg1','sec2','reg2','gdp']]
        proxy_trade.columns = ['year','sector','region','sector','region','gdp']
        proxy_trade.to_csv(os.path.join(data_path,'mrio_analysis','proxy_trade_sec{}.csv'.format(sector)),index=False)
    
    else:
        proxy_trade = pd.DataFrame(columns=['year','gdp'],index= mi_index).reset_index()
        proxy_trade['year'] = 2016
        proxy_trade['gdp'] = 0
        proxy_trade = proxy_trade.query("reg1 != reg2")    
        proxy_trade = proxy_trade.loc[proxy_trade.sec1 == sector]
        proxy_trade = proxy_trade.loc[proxy_trade.sec2.isin(['L','M','N','O','P'])]
        proxy_trade['sec1'] = proxy_trade.sec1.apply(change_name)
        proxy_trade['sec2'] = proxy_trade.sec2.apply(change_name) 
        
        proxy_trade = proxy_trade.query("reg1 == reg2")    

        proxy_trade = proxy_trade[['year','sec1','reg1','sec2','reg2','gdp']]
        proxy_trade.columns = ['year','sector','region','sector','region','gdp']

        proxy_trade.to_csv(os.path.join(data_path,'mrio_analysis','proxy_trade_{}.csv'.format(sector)),index=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:02<00:00,  7.16it/s]


In [194]:
%%time
p = subprocess.Popen(['D:\Dropbox\OIA\Argentina\Data\mrio_analysis\mrio_disaggregate', 'settings_trade.yml'],
                     cwd=os.path.join(data_path, 'mrio_analysis'))
p.wait()

Wall time: 29.7 s


In [195]:
region_names_list = [item for sublist in [[x]*(len(sectors)+2) for x in region_names]
                     for item in sublist]

rows = ([x for x in sectors+['VA','IMP']])*len(region_names)
cols = ([x for x in sectors+['FD','EXP']])*len(region_names)

index_mi = pd.MultiIndex.from_arrays([region_names_list, rows], names=('region', 'row'))
column_mi = pd.MultiIndex.from_arrays([region_names_list, cols], names=('region', 'col'))

MRIO = pd.read_csv(os.path.join(data_path,'mrio_analysis','output2.csv'),header=None,index_col=None)
MRIO.index = index_mi
MRIO.columns = column_mi

# create predefined index and col, which is easier to read
sector_only = [x for x in sectors]*len(region_names)
col_only = ['FD','EXP']*len(region_names)

region_col = [item for sublist in [[x]*len(sectors) for x in region_names] for item in sublist] + \
    [item for sublist in [[x]*2 for x in region_names] for item in sublist]

column_mi_reorder = pd.MultiIndex.from_arrays(
    [region_col, sector_only+col_only], names=('region', 'col'))

# sum va and imports
valueA = pd.DataFrame(MRIO.loc[MRIO.index.get_level_values(1) == 'VA'].sum(axis='index'))
valueA.columns = pd.MultiIndex.from_product([['Total'],['ValueA']],names=['region','row'])

IMP = pd.DataFrame(MRIO.loc[MRIO.index.get_level_values(1) == 'IMP'].sum(axis='index'))
IMP.columns = pd.MultiIndex.from_product([['Total'],['IMP']],names=['region','row'])

output = pd.concat([MRIO.loc[~MRIO.index.get_level_values(1).isin(['FD','EXP'])]])
output = output.drop(['VA','IMP'], level=1)
output = pd.concat([output,valueA.T,IMP.T])

output = output.reindex(column_mi_reorder, axis='columns')

In [196]:
prov_ratios = pd.DataFrame((prov_data.iloc[:16,:24].stack().swaplevel(i=-2,
             j=-1)/sum(prov_data.iloc[:16,:24].stack().swaplevel(i=-2, 
             j=-1))),columns=['ratio']).reset_index().groupby(['level_0','level_1']).sum().reindex(output.index)[:384]

In [197]:
output_ratio = pd.DataFrame((output.sum(level=0,axis=1).sum(axis=1)[:-2]/sum(output.sum(level=0,axis=1).sum(axis=1)[:-2])),columns=['ratio'])

In [198]:
new_values = pd.DataFrame(output.sum(level=0,axis=1).sum(axis=1)[:-2],columns=['ratio']).multiply(prov_ratios).divide(output_ratio).fillna(0)

In [199]:
%%time
#mrio_arg = ras_method(np.array(output).T,np.array(list(new_values.ratio)+list(output.sum(axis=0)[-48:])),
#                      np.array(list(new_values.ratio)+[output.sum(axis=0)[386].sum(),output.sum(axis=0)[387].sum()]), eps=1e-10,print_out=True)
mrio_arg = ras_method(np.array(output).T,np.array(list(new_values.ratio)+list(output.sum(axis=0)[-48:])),
                      np.array(list(new_values.ratio)+[6948,986]), 
                      eps=1e-3,print_out=False)

0.0024810031546280698
0.002498837327948422
0.0025167996986397156
0.0025348911881906133
0.0025531127245130847
0.0025714652423403095
0.0025899496835677382
0.002608566996514128
0.0026273181359215414
0.0026462040640353734
0.0026652257498653853
0.002684384169015175
0.002703680304307454
0.0027231151458977365
0.002742689690990119
0.002762404943553065
0.0027822619148878402
0.0028022616241969445
0.002822405096935654
0.0028426933670289145
0.002863127475222882
0.002883708469425983
0.0029044374060163136
0.002925315348136337
0.002946343367057125
0.002967522541496237
0.0029888539580724682
0.003010338711192162
0.003031977902651306
0.003053772643170305
0.0030757240504613037
0.0030978332507629602
0.0031201013787267584
0.003142529576336983
0.003165118994502336
0.003187870791975911
0.0032107861357530965
0.0032338662018105424
0.003257112174253507
0.0032805252453158573
0.0033041066164400945
0.003327857497538389
0.0033517791065946767
0.0033758726711994314
0.0034001394273559526
0.003424580620276174
0.0034491

In [200]:
mrio_arg = ras_method(np.array(output).T,np.array(list(output.sum(axis=1))[:384]+list(output.sum(axis=0)[-48:])),
                      np.array(list(output.sum(axis=1))[:384]+[6948,986]), 
                      eps=1e-4,print_out=False)

In [201]:
mrio_argentina = pd.DataFrame(mrio_arg.T,index=output.index,columns=output.columns)

In [202]:
national = mrio_argentina.sum(level=1,axis=0).sum(level=1,axis=1)

In [152]:
mrio_argentina.to_csv('D:\Dropbox\OIA\Argentina\Data\economic_IO_tables\output\mrio_argentina.csv')